In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

1950-2023, 1990-2023.... permet de comparer dans le temps

# Data Mensuelle

In [2]:
df_monthly = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios.CSV', header=6)
df_monthly = df_monthly.rename(columns={'Unnamed: 0': 'Date'})

df_48ind = df_monthly.iloc[:1171].copy()
df_48ind['Date'] = pd.to_datetime(df_48ind['Date'], format='%Y%m')
df_48ind.set_index('Date', inplace=True)
df_48ind = df_48ind.apply(pd.to_numeric, errors='coerce')
df_48ind.replace(-99.99, np.nan, inplace = True)
df_48ind.replace(-999, np.nan, inplace = True)
df_48ind.dropna(inplace = True)

df_numfirm = df_monthly.iloc[2564-20:3735-20].copy()
df_numfirm['Date'] = pd.to_datetime(df_numfirm['Date'], format='%Y%m')
df_numfirm.set_index('Date', inplace=True)
df_numfirm = df_numfirm.apply(pd.to_numeric, errors='coerce')

df_avgsize = df_monthly.iloc[3739-22:4910-22].copy()
df_avgsize['Date'] = pd.to_datetime(df_avgsize['Date'], format='%Y%m')
df_avgsize.set_index('Date', inplace=True)
df_avgsize = df_avgsize.apply(pd.to_numeric, errors='coerce')


df_MC = df_numfirm.multiply(df_avgsize, axis=0)
df_MC = df_MC.loc[df_48ind.index]


df_BtoM = df_monthly.iloc[4890:4988].copy()
df_BtoM = df_BtoM.apply(pd.to_numeric, errors='coerce')

df_BtoM = df_BtoM.loc[df_BtoM.index.repeat(12)].reset_index(drop=True)

df_BtoM['Date'] = pd.to_datetime(df_BtoM['Date'], format='%Y')

df_BtoM = df_BtoM.drop('Date', axis=1)

df_BtoM.replace(-99.99, np.nan, inplace = True)
df_BtoM.replace(-999, np.nan, inplace = True)
df_BtoM.dropna(inplace = True)

df_BtoM = pd.DataFrame(data = df_BtoM.iloc[5:].values, index = df_48ind.index, columns = df_48ind.columns)


df_mom = df_48ind.rolling(window=12).mean()

df_mom.replace(-99.99, np.nan, inplace = True)
df_mom.replace(-999, np.nan, inplace = True)
df_mom.dropna(inplace = True)

In [3]:
df_48ind = df_48ind.loc[df_mom.index]
df_MC = df_MC.loc[df_mom.index]
df_BtoM = df_BtoM.loc[df_mom.index]

In [4]:
df_BtoM

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1970-06-01,1.26,0.65,0.22,0.61,0.57,0.62,0.77,0.67,0.30,0.94,...,0.55,2.29,0.84,0.59,0.50,0.74,0.84,0.65,1.26,1.59
1970-07-01,1.26,0.65,0.22,0.61,0.57,0.62,0.77,0.67,0.30,0.94,...,0.55,2.29,0.84,0.59,0.50,0.74,0.84,0.65,1.26,1.59
1970-08-01,1.26,0.65,0.22,0.61,0.57,0.62,0.77,0.67,0.30,0.94,...,0.55,2.29,0.84,0.59,0.50,0.74,0.84,0.65,1.26,1.59
1970-09-01,1.26,0.65,0.22,0.61,0.57,0.62,0.77,0.67,0.30,0.94,...,0.55,2.29,0.84,0.59,0.50,0.74,0.84,0.65,1.26,1.59
1970-10-01,1.26,0.65,0.22,0.61,0.57,0.62,0.77,0.67,0.30,0.94,...,0.55,2.29,0.84,0.59,0.50,0.74,0.84,0.65,1.26,1.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,0.65,0.43,0.10,0.13,0.28,0.39,0.17,0.59,0.14,0.19,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48
2023-10-01,0.65,0.43,0.10,0.13,0.28,0.39,0.17,0.59,0.14,0.19,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48
2023-11-01,0.65,0.43,0.10,0.13,0.28,0.39,0.17,0.59,0.14,0.19,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48


# Data Journalière

In [5]:
df_daily = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios_Daily.csv', header=5)
df_daily = df_daily.rename(columns={'Unnamed: 0': 'Date'})

df_daily_ret = df_daily.iloc[:25670].copy()
df_daily_ret['Date'] = pd.to_datetime(df_daily_ret['Date'], format='%Y%m%d')
df_daily_ret.set_index('Date', inplace=True)
df_daily_ret = df_daily_ret.apply(pd.to_numeric, errors='coerce')
df_daily_ret.replace(-99.99, np.nan, inplace = True)
df_daily_ret.replace(-999, np.nan, inplace = True)
df_daily_ret.dropna(inplace = True)


df_FF = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/F-F_Research_Data_Factors_daily.CSV', header=3)
df_FF = df_FF.rename(columns={'Unnamed: 0': 'Date'})
df_FF_daily = df_FF.iloc[:25670].copy()
df_FF_daily['Date'] = pd.to_datetime(df_FF_daily['Date'], format='%Y%m%d')
df_FF_daily.set_index('Date', inplace=True)
df_FF_daily = df_FF_daily.apply(pd.to_numeric, errors='coerce')
df_FF_daily.replace(-99.99, np.nan, inplace = True)
df_FF_daily.replace(-999, np.nan, inplace = True)
df_FF_daily.dropna(inplace = True)
df_FF_daily = df_FF_daily.loc[df_daily_ret.index]

/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_3685/2405719038.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_daily = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios_Daily.csv', header=5)


In [6]:
df_daily_ret_reshaped = df_daily_ret.iloc[231:]

df_daily_ret_reshaped


,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1970-06-01,4.26,2.13,3.98,0.62,3.94,2.41,3.47,0.35,1.89,2.14,...,0.80,2.24,3.53,1.74,2.66,1.73,3.57,8.20,0.64,1.74
1970-06-02,2.54,-0.05,-0.83,-1.00,0.02,-0.64,0.70,-0.35,0.63,0.41,...,1.01,-0.30,0.89,-0.19,0.29,0.13,2.79,-1.94,0.20,-2.77
1970-06-03,2.08,0.60,0.92,2.48,1.90,1.79,3.23,2.90,1.18,1.23,...,1.63,0.40,4.51,1.34,2.55,1.03,1.31,3.59,2.19,5.13
1970-06-04,2.60,0.10,0.00,-1.33,0.81,-3.59,-4.59,-0.30,-1.89,-1.29,...,-0.84,-2.14,-3.41,-0.72,-1.33,-0.69,-2.32,-4.62,-1.36,-2.41
1970-06-05,-1.34,-0.71,-2.56,-2.17,-0.36,-1.87,-0.35,-1.90,-1.04,-1.61,...,-1.08,-2.41,-2.67,-1.08,-2.96,-0.90,-3.01,-0.05,-2.08,-2.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-25,-0.08,0.87,0.35,0.86,0.48,1.33,2.14,0.82,1.90,0.43,...,2.30,1.67,0.67,0.74,0.08,0.52,-1.63,1.80,0.78,0.91
2024-01-26,1.13,0.55,0.29,0.67,-0.28,1.11,0.76,0.11,0.59,1.59,...,0.23,-0.46,-0.31,0.76,-0.08,0.42,0.74,0.24,0.02,0.84
2024-01-29,0.04,0.52,0.54,0.22,0.90,0.80,0.81,1.08,0.56,1.13,...,-0.65,0.76,0.70,0.81,1.11,0.87,-0.02,1.42,0.46,-0.29


# Question4

In [7]:
###### une colonne à la fois, que je choisis


Ri_t = df_daily_ret['Agric']
Rft = df_FF_daily['RF']
Rm_t = df_FF_daily['Mkt-RF']

Y = Ri_t - Rft
X = Rm_t

X = sm.add_constant(X)

model = sm.OLS(Y, X)
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.306
Model:                            OLS   Adj. R-squared:                  0.306
Method:                 Least Squares   F-statistic:                     6068.
Date:                Wed, 27 Mar 2024   Prob (F-statistic):               0.00
Time:                        13:43:26   Log-Likelihood:                -22213.
No. Observations:               13767   AIC:                         4.443e+04
Df Residuals:                   13765   BIC:                         4.445e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0093      0.010      0.901      0.3

In [8]:
########## boucle pour toutes les colonnes



Rft = df_FF_daily['RF']
Rm_t = df_FF_daily['Mkt-RF']

betas = []  # Create an empty list to store the betas

for column in df_daily_ret.columns:
    Ri_t = df_daily_ret[column]

    Y = Ri_t - Rft
    X = Rm_t

    X = sm.add_constant(X)

    model = sm.OLS(Y, X)
    results = model.fit()

    betas.append(results.params[1])  # Add the betas to the list

    #print(f"Results for column {column}:")
    #print(results.summary())
    #print("\n---\n")

# Convert the list of betas to a numpy array
betas = np.array(betas)

In [9]:
########## boucle pour toutes les colonnes, avec une date de début et de fin


# Définir la date de début et de fin
start = '1969-06-01'  # Remplacez par la date de début souhaitée
end = '1970-06-01'  # Remplacez par la date de fin souhaitée

# Sélectionner les données pour la plage de dates spécifiée
df_daily_ret_selected = df_daily_ret.loc[start:end]
df_FF_daily_selected = df_FF_daily.loc[start:end]

# Maintenant, vous pouvez utiliser df_daily_ret_selected et df_FF_daily_selected pour votre régression
Rft = df_FF_daily_selected['RF']
Rm_t = df_FF_daily_selected['Mkt-RF']

betas = []  # Create an empty list to store the betas

for column in df_daily_ret_selected.columns:
    Ri_t = df_daily_ret_selected[column]

    Y = Ri_t - Rft
    X = Rm_t

    X = sm.add_constant(X)

    model = sm.OLS(Y, X)
    results = model.fit()

    betas.append(results.params[1])  # Add the betas to the list

    #print(f"Results for column {column}:")
    #print(results.summary())
    #print("\n---\n")

# Convert the list of betas to a numpy array
betas = np.array(betas)
#betas

In [18]:
betas

array([0.76987361, 0.39497047, 0.43484278, 0.43774058, 0.52004467,
       1.38366895, 1.24759136, 1.0128837 , 0.52030488, 1.08036573,
       0.79544396, 0.80031562, 0.51599283, 1.11459073, 1.14245714,
       1.36375867, 1.31398326, 1.18763878, 1.39303143, 2.17577611,
       1.22056013, 1.51922583, 1.89050919, 0.73034197, 0.93837147,
       0.45746986, 0.64166913, 1.43316495, 0.87216487, 0.69466816,
       0.56209365, 0.78857375, 1.08400447, 1.19625632, 1.00509784,
       1.26534414, 1.05765793, 0.74490939, 0.92901845, 1.05579579,
       0.7543746 , 0.97596767, 0.79294898, 1.19904139, 0.50870456,
       1.44389272, 1.2121657 , 0.67106304])

In [10]:
########## boucle pour toutes les colonnes, avec une date de début et de fin, et pour chaque mois


def calculate_betas(start_date, end_date, df_daily_ret_reshaped, df_FF_daily):
    # Sélectionner les données pour la plage de dates spécifiée
    df_daily_ret_selected = df_daily_ret_reshaped.loc[start_date:end_date]
    df_FF_daily_selected = df_FF_daily.loc[start_date:end_date]

    # Maintenant, vous pouvez utiliser df_daily_ret_selected et df_FF_daily_selected pour votre régression
    Rft = df_FF_daily_selected['RF']
    Rm_t = df_FF_daily_selected['Mkt-RF']

    betas = []  # Create an empty list to store the betas

    for column in df_daily_ret_selected.columns:
        Ri_t = df_daily_ret_selected[column]

        Y = Ri_t - Rft
        X = Rm_t

        X = sm.add_constant(X)

        model = sm.OLS(Y, X)
        results = model.fit()

        betas.append(results.params[1])  # Add the betas to the list

    # Convert the list of betas to a numpy array
    betas = np.array(betas)

    return betas

# Get the first and last date in the data
first_date = df_daily_ret_reshaped.index.min()
last_date = df_daily_ret_reshaped.index.max()

# Create a date range for each month in the data
date_range = pd.date_range(start=first_date, end=last_date, freq='M')

# Calculate the betas for each month
monthly_betas = {}
for date in date_range:
    start_date = date - pd.DateOffset(months=12)
    end_date = date
    betas = calculate_betas(start_date, end_date, df_daily_ret, df_FF_daily)
    monthly_betas[date] = betas
    
    # Convert the dictionary to a DataFrame
df_monthly_betas = pd.DataFrame(monthly_betas.items(), columns=['Date', 'Betas'])

# Convert the Betas column from numpy arrays to lists
df_monthly_betas['Betas'] = df_monthly_betas['Betas'].apply(list)

# Set the Date column as the index
df_monthly_betas.set_index('Date', inplace=True)

# Convert each item in the 'Betas' list to a separate column
df_monthly_betas = df_monthly_betas['Betas'].apply(pd.Series)

# Set the index of df_monthly_betas to match df_48ind
df_monthly_betas.index = df_48ind.index

# Set the column names of df_monthly_betas to match df_48ind
df_monthly_betas.columns = df_48ind.columns

df_monthly_betas

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1970-06-01,0.833300,0.667125,0.863092,0.771851,0.826493,1.443119,1.804466,0.843642,1.077667,1.026350,...,0.882323,1.217604,1.538488,0.867460,1.562250,0.939593,1.235038,1.907439,1.224393,1.472233
1970-07-01,0.784879,0.675096,0.843988,0.781457,0.853888,1.428448,1.799682,0.858336,1.085637,1.008964,...,0.888762,1.200762,1.500939,0.864212,1.511359,0.907740,1.211172,1.833603,1.232598,1.328590
1970-08-01,0.762385,0.670398,0.848800,0.783464,0.817538,1.421113,1.766259,0.859772,1.092560,1.039435,...,0.891352,1.193065,1.485241,0.879298,1.531338,0.899930,1.190451,1.778095,1.214517,1.292728
1970-09-01,0.753763,0.659944,0.842075,0.791840,0.785367,1.414583,1.807078,0.882555,1.081722,1.051487,...,0.895087,1.205915,1.488225,0.874983,1.546933,0.865375,1.185750,1.809657,1.216190,1.321286
1970-10-01,0.753830,0.649579,0.850044,0.772374,0.803855,1.401090,1.796751,0.873792,1.090984,1.043327,...,0.886133,1.221189,1.477568,0.870219,1.546773,0.849481,1.193666,1.807833,1.213612,1.359679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,0.525864,0.345300,0.483074,0.400676,0.430589,1.367577,1.280294,0.942431,0.605976,1.204138,...,0.872951,1.026551,0.722212,1.091575,0.724900,1.040178,0.501774,1.336803,1.047362,0.695435
2023-10-01,0.630318,0.370515,0.467977,0.433429,0.472716,1.322775,1.269899,0.972317,0.549883,1.119337,...,0.877257,1.044743,0.768525,1.041089,0.761493,1.055024,0.558025,1.310245,1.073931,0.702216
2023-11-01,0.648529,0.373912,0.425893,0.415404,0.485203,1.357974,1.226310,0.974939,0.518504,1.065300,...,0.908339,1.075861,0.786361,1.043795,0.780124,1.128031,0.526060,1.392576,1.130661,0.688749


# Question5

In [11]:
# Définir la date de début et de fin
start = '1970-06-01'  # Remplacez par la date de début souhaitée
end = '1970-06-30'  # Remplacez par la date de fin souhaitée

# Sélectionner les données pour la plage de dates spécifiée
df_daily_ret_selected = df_daily_ret_reshaped.loc[start:end]
df_FF_daily_selected = df_FF_daily.loc[start:end]

# Maintenant, vous pouvez utiliser df_daily_ret_selected et df_FF_daily_selected pour votre régression
Rft = df_FF_daily_selected['RF']
Rm_t = df_FF_daily_selected['Mkt-RF']
SMB = df_FF_daily_selected['SMB']
HML = df_FF_daily_selected['HML']

volatilities = []  # Create an empty list to store the volatilities

for column in df_daily_ret_selected.columns:
    Ri_t = df_daily_ret_selected[column]

    Y = Ri_t - Rft
    X = pd.concat([Rm_t, SMB, HML], axis=1)

    X = sm.add_constant(X)

    model = sm.OLS(Y, X)
    results = model.fit()

    residuals = results.resid
    volatility = np.std(residuals)

    volatilities.append(volatility)  # Add the volatility to the list

# Convert the list of volatilities to a numpy array
volatilities = np.array(volatilities)

volatilities

array([1.28680306, 0.37116512, 0.94769646, 0.65427336, 0.75393702,
       0.78439333, 0.84452758, 0.87258418, 0.46409757, 0.33271342,
       3.43082805, 0.89625592, 0.31295591, 0.37893134, 0.64915101,
       0.42554039, 0.3234221 , 1.14996565, 0.33465444, 1.38544937,
       0.24439097, 0.56406702, 0.68042019, 0.80981681, 0.56708608,
       0.95968991, 1.13611636, 0.60882783, 2.13591103, 0.32564229,
       0.2532111 , 0.63957981, 0.91321925, 0.70574457, 0.44384281,
       0.55632136, 1.16473921, 0.77504709, 0.47263417, 0.68258681,
       0.63941844, 0.39294691, 0.67131088, 0.56248466, 0.73978957,
       1.31915599, 0.53334247, 1.58179797])

In [19]:
def calculate_idiosyncratic_volatility(start_date, end_date, df_daily_ret_reshaped, df_FF_daily):
    # Sélectionner les données pour la plage de dates spécifiée
    df_daily_ret_selected = df_daily_ret_reshaped.loc[start_date:end_date]
    df_FF_daily_selected = df_FF_daily.loc[start_date:end_date]

    # Maintenant, vous pouvez utiliser df_daily_ret_selected et df_FF_daily_selected pour votre régression
    Rft = df_FF_daily_selected['RF']
    Rm_t = df_FF_daily_selected['Mkt-RF']
    SMB = df_FF_daily_selected['SMB']
    HML = df_FF_daily_selected['HML']

    volatilities = []  # Create an empty list to store the volatilities

    for column in df_daily_ret_selected.columns:
        Ri_t = df_daily_ret_selected[column]

        Y = Ri_t - Rft
        X = pd.concat([Rm_t, SMB, HML], axis=1)

        X = sm.add_constant(X)

        model = sm.OLS(Y, X)
        results = model.fit()

        residuals = results.resid
        volatility = np.std(residuals)

        volatilities.append(volatility)  # Add the volatility to the list

    # Convert the list of volatilities to a numpy array
    volatilities = np.array(volatilities)

    return volatilities

# Get the first and last date in the data
first_date = df_daily_ret_reshaped.index.min()
last_date = df_daily_ret_reshaped.index.max()

# Create a date range for each month in the data
date_range = pd.date_range(start=first_date, end=last_date, freq='M')

# Calculate the volatilities for each month
monthly_volatilities = {}
for date in date_range:
    start_date = date  - pd.DateOffset(months=1) + pd.DateOffset(days=1)
    end_date = date
    volatilities = calculate_idiosyncratic_volatility(start_date, end_date, df_daily_ret, df_FF_daily)
    monthly_volatilities[date] = volatilities
    
        # Convert the dictionary to a DataFrame
df_monthly_vol = pd.DataFrame(monthly_volatilities.items(), columns=['Date', 'Betas'])

# Convert the Betas column from numpy arrays to lists
df_monthly_vol['Betas'] = df_monthly_vol['Betas'].apply(list)

# Set the Date column as the index
df_monthly_vol.set_index('Date', inplace=True)

# Convert each item in the 'Betas' list to a separate column
df_monthly_vol = df_monthly_vol['Betas'].apply(pd.Series)

# Set the index of df_monthly_vol to match df_48ind
df_monthly_vol.index = df_48ind.index

# Set the column names of df_monthly_vol to match df_48ind
df_monthly_vol.columns = df_48ind.columns

df_monthly_vol

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1970-06-01,1.286803,0.371165,0.947696,0.654273,0.753937,0.784393,0.844528,0.872584,0.464098,0.332713,...,0.472634,0.682587,0.639418,0.392947,0.671311,0.562485,0.739790,1.319156,0.533342,1.581798
1970-07-01,0.903645,0.297988,0.983878,0.539850,1.118638,0.867436,1.024037,0.654378,0.471037,0.432825,...,0.624610,0.605158,0.562184,0.448060,0.928359,0.383881,0.492577,1.009767,0.574494,2.014950
1970-08-01,1.286118,0.228737,0.560716,0.493485,0.506995,0.433123,1.024717,0.538519,0.311973,0.546849,...,0.540441,0.520265,0.481653,0.326068,0.769190,0.271083,0.288478,1.126670,0.374807,2.216611
1970-09-01,0.888272,0.265451,0.685087,0.392908,0.509686,0.610811,1.272241,0.612354,0.251277,0.387732,...,0.625137,0.735453,0.493105,0.329702,0.910135,0.392863,0.504881,0.861496,0.408002,1.660003
1970-10-01,0.861061,0.249272,0.882918,0.578580,0.725742,0.586638,0.662836,0.517358,0.375591,0.323266,...,0.395832,0.804666,0.313766,0.209945,0.663697,0.232661,0.700286,0.797523,0.442886,1.194961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,0.879675,0.594519,0.505139,0.620083,0.656747,0.698070,0.679948,0.754514,0.541715,1.090389,...,0.406030,0.387055,0.464615,0.548382,0.514544,0.336849,0.643869,1.050746,0.325864,0.482346
2023-10-01,0.889751,1.049251,1.330358,1.337005,1.087573,1.288871,1.846620,0.564938,0.825623,0.864228,...,1.140727,0.709256,0.343181,0.776928,0.663266,0.452632,0.629312,0.529945,0.481207,0.395495
2023-11-01,1.689746,0.510963,0.437368,0.517795,0.536647,0.797260,0.553298,0.734844,0.626014,0.904951,...,0.700502,0.345972,0.453672,0.522812,0.482865,0.277303,0.643216,0.826251,0.407948,0.330827


df_sorted = df_48ind.loc[date].sort_values()

In [13]:
def calculate_portfolio_return(df, num_positions, df_returns):
    # Pour chaque mois et chaque caractéristique
    portfolio_returns = []
    for date in df.index:
        # Tri des portefeuilles
        df_sorted = df.loc[date].sort_values()

        # Création des positions longues et courtes
        long_positions = df_sorted.tail(num_positions)
        short_positions = df_sorted.head(num_positions)

        # Calcul des poids (ici, nous supposons que les poids sont équipondérés)
        weights_long = [1/num_positions] * num_positions
        weights_short = [1/num_positions] * num_positions

        # Calcul du rendement du portefeuille
        # Nous supposons ici que vous avez un autre DataFrame 'df_returns' qui contient les rendements
        portfolio_return = (df_returns.loc[date, long_positions.index] * weights_long).sum() - (df_returns.loc[date, short_positions.index] * weights_short).sum()

        portfolio_returns.append(portfolio_return)

    return portfolio_returns

In [14]:
def select_extreme_values(row, num_values=5):
    sorted_row = row.sort_values(ascending=False)
    top_values = sorted_row.head(num_values)
    bottom_values = sorted_row.tail(num_values)
    return top_values, bottom_values

In [15]:
top_bottom_values = df_MC.apply(select_extreme_values, axis=1)

top_bottom_values

Date
1970-06-01    ([60334.82, 41540.4, 39881.81, 39247.56, 33573...
1970-07-01    ([60376.14, 40260.479999999996, 38423.7, 35565...
1970-08-01    ([67903.11, 42074.88, 41604.3, 35711.69, 33617...
1970-09-01    ([71867.06999999999, 43743.36, 43281.0, 38254....
1970-10-01    ([73925.32, 46452.479999999996, 42871.95, 4226...
                                    ...                        
2023-09-01    ([8962789.42, 6024688.6, 3359902.26, 2954610.7...
2023-10-01    ([8484942.42, 5517336.7299999995, 3152332.77, ...
2023-11-01    ([8434209.78, 5356977.84, 3174484.6, 2711854.0...
2023-12-01    ([9459383.58, 6026246.399999999, 3390874.19999...
2024-01-01    ([9792394.24, 6392489.87, 3597482.63, 3017472....
Length: 644, dtype: object

In [16]:
print(df_MC.iloc[0])

Agric      223.52
Food     13989.30
Soda      5965.57
Beer      2888.25
Smoke     3756.90
Toys      2921.60
Fun       1584.44
Books     2492.05
Hshld    41540.40
Clths     2809.08
Hlth        76.78
MedEq     5341.35
Drugs    21970.24
Chems    19217.52
Rubbr      558.24
Txtls     2826.72
BldMt    12519.87
Cnstr      902.16
Steel    14923.09
FabPr      501.90
Mach     12195.86
ElcEq     5236.54
Autos    31768.87
Aero      3458.39
Ships      898.92
Guns       467.28
Gold       826.70
Mines     4012.47
Coal       691.60
Oil      60334.82
Util     39247.56
Telcm    33573.54
PerSv      271.46
BusSv     3777.20
Comps    39881.81
Chips    11257.92
LabEq     2161.32
Paper     8274.42
Boxes     8814.16
Trans     8838.75
Whlsl     1654.95
Rtail    25291.28
Meals     2712.92
Banks     9080.80
Insur     4932.30
RlEst      872.85
Fin       7903.80
Other       61.08
Name: 1970-06-01 00:00:00, dtype: float64


In [17]:
def get_returns(top_bottom_values, df_48ind):
    returns = []
    for date, values in top_bottom_values.iteritems():
        top_indices, bottom_indices = values[0].index, values[1].index
        top_returns = df_48ind.loc[date, top_indices]
        bottom_returns = df_48ind.loc[date, bottom_indices]
        returns.append((top_returns, bottom_returns))
    return returns

returns = get_returns(top_bottom_values, df_48ind)

AttributeError: 'Series' object has no attribute 'iteritems'

In [ ]:
def get_returns(top_bottom_values, df_48ind):
    returns = []
    for date, values in top_bottom_values.items():
        top_indices, bottom_indices = values[0].index, values[1].index
        top_returns = df_48ind.loc[date, top_indices]
        bottom_returns = df_48ind.loc[date, bottom_indices]
        returns.append((top_returns, bottom_returns))
    return returns

returns = get_returns(top_bottom_values, df_48ind)

In [ ]:
returns

[(Oil      -2.44
  Hshld    -5.05
  Comps   -11.75
  Util     -4.70
  Telcm    -8.48
  Name: 1970-06-01 00:00:00, dtype: float64,
  Guns    -10.68
  PerSv   -17.82
  Agric    -0.75
  Hlth     -4.57
  Other   -22.57
  Name: 1970-06-01 00:00:00, dtype: float64),
 (Oil      12.83
  Hshld     4.60
  Util      8.70
  Comps     0.80
  Autos     8.93
  Name: 1970-07-01 00:00:00, dtype: float64,
  FabPr    12.65
  Guns     14.75
  Agric    -0.81
  Hlth     -6.29
  Other     0.12
  Name: 1970-07-01 00:00:00, dtype: float64),
 (Oil      6.64
  Hshld    4.22
  Util     4.67
  Comps    7.14
  Telcm    2.51
  Name: 1970-08-01 00:00:00, dtype: float64,
  FabPr     9.38
  Guns      7.20
  Agric    -0.31
  Hlth      8.96
  Other    14.45
  Name: 1970-08-01 00:00:00, dtype: float64),
 (Oil       2.98
  Hshld     6.38
  Util     -0.52
  Comps    10.53
  Autos     1.36
  Name: 1970-09-01 00:00:00, dtype: float64,
  FabPr    13.03
  Guns      5.97
  Agric    14.45
  Hlth     36.41
  Other     5.75
  Name:

In [ ]:
def get_weighted_returns_ew(top_bottom_values, df_48ind, num_positions):
    returns = []
    weight = 1.0 / num_positions
    for date, values in top_bottom_values.items():
        top_indices, bottom_indices = values[0].index, values[1].index
        top_returns = df_48ind.loc[date, top_indices] * weight
        bottom_returns = df_48ind.loc[date, bottom_indices] * weight * -1  # short positions have negative weight
        returns.append((top_returns.sum(), bottom_returns.sum()))
    return returns

returns = get_weighted_returns_ew(top_bottom_values, df_48ind, 5)

In [ ]:
def get_total_returns_ew(top_bottom_values, df_48ind, num_positions):
    returns = []
    weight = 1.0 / num_positions
    for date, values in top_bottom_values.items():
        top_indices, bottom_indices = values[0].index, values[1].index
        top_returns = df_48ind.loc[date, top_indices] * weight
        bottom_returns = df_48ind.loc[date, bottom_indices] * weight * -1  # short positions have negative weight
        total_return = top_returns.sum() + bottom_returns.sum()
        returns.append(total_return)
    return returns

ret_ew = get_total_returns_ew(top_bottom_values, df_48ind, 5)

In [ ]:
def get_total_returns_vw(top_bottom_values, df_48ind):
    returns = []
    for date, values in top_bottom_values.items():
        top_indices, bottom_indices = values[0].index, values[1].index
        top_values = df_48ind.loc[date, top_indices]
        bottom_values = df_48ind.loc[date, bottom_indices]
        
        top_weights = top_values.abs() / top_values.abs().sum()
        bottom_weights = bottom_values.abs() / bottom_values.abs().sum()
        
        top_returns = top_values * top_weights
        bottom_returns = bottom_values * bottom_weights * -1  # short positions have negative weight
        
        total_return = top_returns.sum() + bottom_returns.sum()
        returns.append(total_return)
    return returns

ret_vw = get_total_returns_vw(top_bottom_values, df_48ind)

In [ ]:
ret_vw

[8.9397607435395,
 -0.21247801080200723,
 -5.041356084331771,
 -16.00545159300588,
 14.979854059962113,
 0.4005756116222967,
 -4.308116028442706,
 -12.059331150862768,
 -16.040858005812115,
 -3.24684716079531,
 2.5994066051648628,
 3.8981943240791077,
 4.566291658737217,
 5.176152762180141,
 0.05116453893714912,
 -0.9598811857875745,
 3.696292406147456,
 5.904008069419112,
 -4.890292203612196,
 -6.115628818018388,
 -3.4008612334202573,
 -4.528037528161924,
 4.689992445366262,
 7.63088129074246,
 6.164405255031317,
 6.678752252120727,
 1.5927021658483174,
 11.079590172597417,
 5.532424206167627,
 -5.819120110763303,
 5.70050677435138,
 0.4802472722638632,
 4.382574789982681,
 6.694106006667936,
 7.446718773870424,
 14.983064816793224,
 8.963872521122449,
 -14.507605703990496,
 2.2803444144953584,
 -18.198036745951875,
 4.157820249739229,
 6.053069719944682,
 -3.0150566073926024,
 -13.910194914237739,
 -10.591600015987355,
 -3.00098210719572,
 0.3696984762226858,
 1.9090798495284451,
 5.